In [89]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import matplotlib.pyplot as plt

In [90]:
def extract_data(annotation_file):

    #import file into pandas dataframe
    anno = pd.read_csv(annotation_file, sep="\t", header=None)

    #label columns correctly
    anno = anno.rename(columns={0: "x1", 1: "y1", 2: "x2", 3: "y2",4: "x3", 5: "y3",6: "x4", 7: "y4",})

    #check there is all 10 rows
    if anno.shape[0] != 10:
        print("ERROR!")

    #remove any extra Nan columns from txt file
    if anno.shape[1] > 8:
        extra_columns = anno.shape[1] - 8
        for i in range(extra_columns):
            anno = anno.drop(columns=[i+8])

    return anno



In [91]:
def calculate_centres(anno,patient_view):

    num_centres = len(anno)

    chosen_anno = anno.iloc[0]
    centres_np = np.empty([1,num_centres*2])
    
    for i in range(len(anno)):

        chosen_anno = anno.iloc[i]

        if np.isnan(chosen_anno[0]) == 0:

            x13 = np.array([chosen_anno[0],chosen_anno[4]])
            y13 = np.array([chosen_anno[1],chosen_anno[5]])

            A13 = np.vstack([x13, np.ones(len(x13))]).T

            m13, c13 = np.linalg.lstsq(A13, y13, rcond=None)[0]


            x24 = np.array([chosen_anno[2],chosen_anno[6]])
            y24 = np.array([chosen_anno[3],chosen_anno[7]])

            A24 = np.vstack([x24, np.ones(len(x24))]).T

            m24, c24 = np.linalg.lstsq(A24, y24, rcond=None)[0]
            xi = (c24-c13) / (m13-m24)
            yi = m13 * xi + c13
            #plt.plot(chosen_anno[::2], chosen_anno[1::2], 'o', label='Original data', markersize=10)
            #plt.plot(x24, m24*x24 + c24, 'r', label='Fitted line 2')
            #plt.plot(x13, m13*x13 + c13, 'r', label='Fitted line 1')
            #plt.plot(xi, yi, 'o', label='Intersection', markersize=10)

            #plt.legend()
            #plt.show()
            
            centres_np[0,2*i] = xi
            centres_np[0,2*i+1] = yi

        else:
            centres_np[0,2*i] = np.nan
            centres_np[0,2*i+1] = np.nan

    centres_df = df(data=centres_np, index=[patient_view], columns=["x-LAD/LCx","y-LAD/LCx", \
                                           "x-Diag1","y-Diag1",\
                                           "x-Diag2","y-Diag2",\
                                           "x-Sep1","y-Sep1",\
                                           "x-Sep2","y-Sep2",\
                                           "x-Diag3","y-Diag3",\
                                           "x-Diag4","y-Diag4",\
                                           "x-Sep3","y-Sep3",\
                                           "x-Sep4","y-Sep4",\
                                           "x-Diag1Bif","y-Diag1Bif"])

    return centres_df

In [92]:
import os
rootdir = '/content/drive/MyDrive/4YPTest/Dataset'

files_to_visit = np.empty([])
zerooverwrite = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        currpath = os.path.join(subdir, file)
        if currpath[-4:] == ".txt":
            if zerooverwrite == 0:
                files_to_visit = currpath
                zerooverwrite = 1
            else:
                files_to_visit = np.append(files_to_visit,currpath)

files_to_visit = np.sort(files_to_visit)

In [97]:
#Put centres into dataframe

#IF CHECKING NO EXTRA ANNOTATIONS
#filestocheck = np.empty([])

for i in range(len(files_to_visit)):
    #try:
        annotation_file = files_to_visit[i]
        anno = extract_data(annotation_file)
        if len(annotation_file) == 67:
            patient_view = annotation_file[47:51]+annotation_file[55]
        else:
            patient_view = annotation_file[47:51]+annotation_file[55:57]
        centresnew = calculate_centres(anno,patient_view)
        if i == 0:
            centres = centresnew
        else:
            centres = pd.concat([centres, centresnew], axis=0)
    #except:
        #filestocheck = np.append(filestocheck, annotation_file)

In [116]:
centres.to_csv('/content/drive/MyDrive/4YPTest/Dataset/centres.csv')